In [48]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import jieba
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import json
import plotly

# 设置matplotlib支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [2]:
def load_character_list(filename):
    """
    从文件中加载人物列表
    Args:
        filename: 人物列表文件名
    Returns:
        character_list: 人物列表
    """
    character_list = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            name = line.strip()
            if name:  # 跳过空行
                character_list.append(name)
    print(f"已加载 {len(character_list)} 个人物")
    return character_list

In [3]:
def add_characters_to_jieba(character_list):
    """
    将人物名字添加到jieba词典中
    Args:
        character_list: 人物列表
    """
    for char in character_list:
        jieba.add_word(char)
    print("已添加人物名字到jieba词典")

In [4]:
def read_and_tokenize_novels(novel_dir):
    """
    读取所有小说并分词
    Args:
        novel_dir: 小说文件所在目录
    Returns:
        sentences: 分词后的句子列表
    """
    sentences = []
    for filename in os.listdir(novel_dir):
        if filename.endswith('.txt'):
            print(f"正在处理文件: {filename}")
            with open(os.path.join(novel_dir, filename), 'r', encoding='utf-8') as f:
                text = f.read()
                # 按句子分割
                for line in text.split('\n'):
                    if line.strip():
                        words = list(jieba.cut(line.strip()))
                        sentences.append(words)
    print(f'处理完成，共处理了 {len(sentences)} 个句子')
    return sentences

In [5]:
def train_word2vec(sentences):
    """
    训练word2vec模型
    Args:
        sentences: 分词后的句子列表
    Returns:
        model: 训练好的word2vec模型
    """
    print("开始训练word2vec模型...")
    model = Word2Vec(
        sentences=sentences,
        vector_size=100,  # 词向量维度
        window=5,         # 窗口大小
        min_count=5,      # 最小词频
        workers=4,        # 使用4个CPU核心
        epochs=10         # 训练轮数
    )
    
    # 保存模型
    model.save('jinyong_word2vec.model')
    print("模型训练完成并已保存")
    return model

In [21]:
def extract_character_vectors(model, character_list):
    """
    提取人物词向量
    Args:
        model: 训练好的word2vec模型
        character_list: 人物列表
    Returns:
        vectors: 词向量数组
        names: 人物名字列表
    """
    character_vectors = {}
    for char in character_list:
        if char in model.wv:
            character_vectors[char] = model.wv[char]
    
    print(f"成功提取了 {len(character_vectors)} 个人物的词向量")
    
    # 转换为numpy数组
    vectors = np.array(list(character_vectors.values()))
    names = list(character_vectors.keys())
    
    return vectors, names

In [22]:
def visualize_with_matplotlib(vectors, names):
    """
    使用matplotlib进行可视化
    Args:
        vectors: 词向量数组
        names: 人物名字列表
    """
    # 进行PCA降维到2维
    pca_2d = PCA(n_components=2)
    vectors_2d = pca_2d.fit_transform(vectors)
    
    # 进行PCA降维到3维
    pca_3d = PCA(n_components=3)
    vectors_3d = pca_3d.fit_transform(vectors)
    
    # 2D可视化
    plt.figure(figsize=(15, 10))  # 增大图形尺寸
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
    for i, name in enumerate(names):
        plt.annotate(name, (vectors_2d[i, 0], vectors_2d[i, 1]), fontsize=8)  # 设置字体大小
    plt.title('金庸小说人物词向量2D可视化', fontsize=14)
    plt.xlabel('第一主成分', fontsize=12)
    plt.ylabel('第二主成分', fontsize=12)
    plt.savefig('characters_2d.png', dpi=300, bbox_inches='tight')  # 提高DPI并调整边界
    plt.close()
    
    # 3D可视化
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(vectors_3d[:, 0], vectors_3d[:, 1], vectors_3d[:, 2])
    for i, name in enumerate(names):
        ax.text(vectors_3d[i, 0], vectors_3d[i, 1], vectors_3d[i, 2], name, fontsize=8)
    ax.set_title('金庸小说人物词向量3D可视化', fontsize=14)
    ax.set_xlabel('第一主成分', fontsize=12)
    ax.set_ylabel('第二主成分', fontsize=12)
    ax.set_zlabel('第三主成分', fontsize=12)
    plt.savefig('characters_3d.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("matplotlib可视化完成，结果已保存为characters_2d.png和characters_3d.png")

In [23]:
def generate_p5js_data(vectors, names):
    """
    生成p5.js可视化所需的数据
    Args:
        vectors: 词向量数组
        names: 人物名字列表
    """
    # 进行PCA降维到3维
    pca_3d = PCA(n_components=3)
    vectors_3d = pca_3d.fit_transform(vectors)
    
    # 保存数据
    data = {
        'vectors': vectors_3d.tolist(),
        'names': names
    }
    with open('character_vectors.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print("p5.js数据生成完成，可以打开visualization.html查看交互式可视化效果")

In [24]:
def generate_p5js_data(vectors, names):
    """
    生成p5.js可视化所需的数据
    Args:
        vectors: 词向量数组
        names: 人物名字列表
    """
    # 进行PCA降维到3维
    pca_3d = PCA(n_components=3)
    vectors_3d = pca_3d.fit_transform(vectors)
    
    # 保存数据
    data = {
        'vectors': vectors_3d.tolist(),
        'names': names
    }
    with open('character_vectors.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print("p5.js数据生成完成，可以打开visualization.html查看交互式可视化效果")

In [36]:
def generate_p5js_data(vectors, names):
    """
    生成p5.js可视化所需的数据
    Args:
        vectors: 词向量数组
        names: 人物名字列表
    """
    # 进行PCA降维到3维
    pca_3d = PCA(n_components=3)
    vectors_3d = pca_3d.fit_transform(vectors)
    
    # 保存数据
    data = {
        'vectors': vectors_3d.tolist(),
        'names': names
    }
    with open('character_vectors.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print("p5.js数据生成完成，可以打开visualization.html查看交互式可视化效果")

In [41]:
def create_p5js_html(vectors, names):
    """
    创建p5.js可视化HTML文件
    Args:
        vectors: 词向量数组
        names: 人物名字列表
    """
    # 将数据转换为JavaScript格式
    vectors_js = json.dumps(vectors.tolist(), ensure_ascii=False)
    names_js = json.dumps(names, ensure_ascii=False)
    
    html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>金庸小说人物关系3D可视化</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/1.4.0/p5.js"></script>
    <style>
        body {{
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f0f0f0;
        }}
        canvas {{
            display: block;
        }}
    </style>
</head>
<body>
    <script>
        // 直接嵌入数据
        let characters = {names_js};
        let vectors = {vectors_js};
        let rotationX = 0;
        let rotationY = 0;
        let isDragging = false;
        let lastMouseX, lastMouseY;
        let zoom = 1;

        function setup() {{
            createCanvas(800, 600, WEBGL);
            textSize(12);
            textAlign(CENTER, CENTER);
        }}

        function draw() {{
            background(240);
            
            rotateX(rotationX);
            rotateY(rotationY);
            scale(zoom);

            // 绘制坐标轴
            stroke(150);
            line(-200, 0, 0, 200, 0, 0); // X轴
            line(0, -200, 0, 0, 200, 0); // Y轴
            line(0, 0, -200, 0, 0, 200); // Z轴

            // 绘制点和标签
            for (let i = 0; i < vectors.length; i++) {{
                let x = vectors[i][0] * 100;
                let y = vectors[i][1] * 100;
                let z = vectors[i][2] * 100;

                push();
                translate(x, y, z);
                noStroke();
                fill(0, 0, 255);
                sphere(5);
                pop();

                push();
                translate(x, y, z);
                fill(0);
                noStroke();
                text(characters[i], 10, 0);
                pop();
            }}
        }}

        function mousePressed() {{
            isDragging = true;
            lastMouseX = mouseX;
            lastMouseY = mouseY;
        }}

        function mouseReleased() {{
            isDragging = false;
        }}

        function mouseDragged() {{
            if (isDragging) {{
                let deltaX = mouseX - lastMouseX;
                let deltaY = mouseY - lastMouseY;
                rotationY += deltaX * 0.01;
                rotationX += deltaY * 0.01;
                lastMouseX = mouseX;
                lastMouseY = mouseY;
            }}
        }}

        function mouseWheel(event) {{
            zoom += event.delta * 0.001;
            zoom = constrain(zoom, 0.1, 3);
        }}
    </script>
</body>
</html>
"""
    with open('visualization.html', 'w', encoding='utf-8') as f:
        f.write(html_content)
    print("已生成visualization.html文件")

In [44]:
def visualize_with_plotly(vectors, names):
    """
    使用plotly进行交互式可视化
    Args:
        vectors: 词向量数组
        names: 人物名字列表
    """
    import plotly.graph_objects as go
    
    # 进行PCA降维到3维
    pca_3d = PCA(n_components=3)
    vectors_3d = pca_3d.fit_transform(vectors)
    
    # 创建3D散点图
    fig = go.Figure(data=[go.Scatter3d(
        x=vectors_3d[:, 0],
        y=vectors_3d[:, 1],
        z=vectors_3d[:, 2],
        mode='markers+text',
        text=names,
        textposition='top center',
        marker=dict(
            size=8,
            color='blue',
            opacity=0.8
        ),
        textfont=dict(
            size=12,
            color='black'
        )
    )])
    
    # 更新布局
    fig.update_layout(
        title='金庸小说人物词向量3D可视化',
        scene=dict(
            xaxis_title='第一主成分',
            yaxis_title='第二主成分',
            zaxis_title='第三主成分',
            xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'),
            yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'),
            zaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray')
        ),
        width=1000,
        height=800,
        margin=dict(l=0, r=0, b=0, t=30)
    )
    
    # 保存为HTML文件
    fig.write_html('characters_plotly.html')
    print("plotly交互式可视化完成，可以打开characters_plotly.html查看效果")

In [38]:
# 加载人物列表
character_list = load_character_list('characters_names_full.txt')

已加载 1373 个人物


In [27]:
# 添加人物到jieba词典
add_characters_to_jieba(character_list)

已添加人物名字到jieba词典


In [28]:
# 读取并处理小说
sentences = read_and_tokenize_novels('novels')

正在处理文件: 金庸01飞狐外传.txt
正在处理文件: 金庸02雪山飞狐.txt
正在处理文件: 金庸03连城诀.txt
正在处理文件: 金庸04天龙八部.txt
正在处理文件: 金庸05射雕英雄传.txt
正在处理文件: 金庸06白马啸西风.txt
正在处理文件: 金庸07鹿鼎记.txt
正在处理文件: 金庸08笑傲江湖.txt
正在处理文件: 金庸09书剑恩仇录.txt
正在处理文件: 金庸10神雕侠侣.txt
正在处理文件: 金庸11侠客行.txt
正在处理文件: 金庸12倚天屠龙记.txt
正在处理文件: 金庸13碧血剑.txt
正在处理文件: 金庸14鸳鸯刀.txt
正在处理文件: 金庸15越女剑.txt
处理完成，共处理了 47441 个句子


In [18]:
# 训练模型
model = train_word2vec(sentences)  

开始训练word2vec模型...
模型训练完成并已保存


In [29]:
# 提取词向量
vectors, names = extract_character_vectors(model, character_list)

成功提取了 1203 个人物的词向量


In [30]:
# matplotlib可视化
visualize_with_matplotlib(vectors, names)

matplotlib可视化完成，结果已保存为characters_2d.png和characters_3d.png


In [43]:
# p5.js可视化
generate_p5js_data(vectors, names)
create_p5js_html(vectors, names)

p5.js数据生成完成，可以打开visualization.html查看交互式可视化效果
已生成visualization.html文件


In [49]:
# plotly交互式可视化
visualize_with_plotly(vectors, names)

plotly交互式可视化完成，可以打开characters_plotly.html查看效果
